<a href="https://colab.research.google.com/github/nyoons/ESAA/blob/main/Ch09_%EB%B9%84%EC%A7%80%EB%8F%84_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Ch 09. 비지도 학습**

대부분 머신러닝이 지도 학습 기반이지만 사용할 수 있는 데이터는 대부분 레이블이 없다.

제품 사진 예시. 제품 사진을 찍는건 쉬우나 여기엔 레이블이 없다. 결함 또는 정상이라고 레이블을 부여해야 한다. 수동으로 하면 시간이 문제. 그래서 레이블 된 데이터셋은 매우 작다는 문제가 생김. 레이블을 붙일 필요 없이 알고리즘이 레이블이 없는 데이터를 바로 사용하면? 비지도 학습이 필요.

비지도 학습이 대표 차원 축소. 몇가지 비지도 학습과 알고리즘을 추가로 알아보자.

- 군집 : 비슷한 샘플을 클러스터로 모은다. 데분, 고객 분류, 추천 시스템, 검색 엔진, 이미지 분할, 준지도 학습, 차원 축소 등에 사용.
- 이상치 탐지 : 정상 데이터가 어떻게 보이는지 학습하고 비정상 샘플 감지.
- 밀도 추정 : 데이터셋 생성 확률 밀도 함수 추정. 밀도 추정은 이상치 탐지에 널리 사용. 밀도가 낮은 영역은 이상치일 가능성이 높다.

k-평균과 DBSCAN을 사용해 군집부터 시작하고, 가우시안 혼합 모델을 설명하고 이를 어떻게 밀도 추정, 군집, 이상치 탐지에 사용할 수 있는지 알아보자.

##9.1 군집
군집 : 비슷한 샘플을 구별해 하나의 클러스터 또는 비슷한 샘플의 그룹으로 할당하는 작업.

각 샘플은 하나의 그룹에 할당된다. 하지만 분류와 달리 레이블이 없는 비지도 학습인 군집은 분류 알고리즘 사용 불가. 군집 알고리즘이 필요. 특성을 적절히 사용하면 군집 알고리즘이 클러스터를 잘 구분할 것.

- 고객 분류 : 구매이력이나 웹사이트 내 행동 기반 클러스터. -> 추천 시스템
- 데분
- 차원 축소 기법 : 각 클러스터에 대한 샘플의 친화성 측정.
- 이상치 탐지 : 모든 클러스터에 친화성이 낮은 샘플은 이상치일 가능성이 높다. -> 부정 거래 감지
- 준지도 학습 : 군집 수행해 모든 샘플에 레이블 전파하여 이어지는 지도 학습 알고리즘에 필요한 레이블 크게 증가 가능
- 검색 엔진 : 제시된 이미지와 비슷한 이미지 찾아주는 등.
- 이미지 분할 : 색을 기반으로 픽셀을 클러스터로 모은다. 물체의 윤곽 감지해 물체 탐지 및 추적 시스템.

클러스터에 대한 보편적 정의는 없다. 알고리즘이 다르면 다른 종류의 클러스터를 감지. ex) 센트로이드(특정 포인트) 중심으로 모인 샘플 찾기, 샘플이 밀집되어 연속된 영역 찾기 등.

###9.1.1 k-평균
샘플 덩어리가 잘 보이는 레이블이 없는 데이터셋 가정. k-평균은 이런 종류의 데이터셋을 반복 몇 번으로 빠르고 효율적으로 클러스터로 처리 가능. 각 클러스터의 중심을 찾고 가장 가까운 클러스터에 샘플 할당.

In [1]:
#데이터 생성 코드

from sklearn.datasets import make_blobs
import numpy as np


blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [2]:
from sklearn.cluster import KMeans
k=5
kmeans=KMeans(n_clusters=k)
y_pred=kmeans.fit_predict(X)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


알고리즘이 찾을 클러스터 개수 k 지정. 일반적으로 쉬운 일은 아니다.

각 샘플은 다섯 개 클러스터 클러스터 중 하나에 할당. 군집의 레이블=클러스터의 인덱스. 분류의 클래스 레이블과 혼동하지 말 것. KMeans 클래스의 인스턴스는 labels_ 인스턴스 변수에 훈련된 샘플의 레이블을 갖는다.

In [3]:
y_pred

array([4, 0, 1, ..., 3, 1, 0], dtype=int32)

In [4]:
y_pred is kmeans.labels_

True

In [5]:
#이 알고리즘이 찾은 센트로이드 다섯 개
kmeans.cluster_centers_

array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-1.46679593,  2.28585348],
       [-2.79290307,  2.79641063],
       [-2.80037642,  1.30082566]])

새로운 샘플에 가장 가까운 센트로이드의 클러스터 할당

In [6]:
X_new=np.array([[0,2], [3,2], [-3,3], [-3,2.5]])
kmeans.predict(X_new)

array([1, 1, 3, 3], dtype=int32)

클러스터의 결정 경계 그림 -> 보로노이 다이어그램

샘플은 대부분 적절한 클러스터에 잘 할당되었다. 하지만 몇 개는 잘못 부여됨. 실제 k-평균 알고리즘은 클러스터의 크기가 많이 다르면 잘 작동하지 않음. 거리만 고려하기 때문.

하드 군집이라는 샘플을 하나의 클러스터에 할당하는 것보다 클러스터마다 샘플에 점수를 부여하는 것이 유용할 수 있다. = 소프트 군집. 이 점수가 샘플과 센트로이드 사이의 거리가 된다. KMeans 클래스의 transform() 메서드는 샘플과 각 센트로이드 사이의 거리를 반환.

In [7]:
kmeans.transform(X_new)

array([[2.81093633, 0.32995317, 1.49439034, 2.9042344 , 2.88633901],
       [5.80730058, 2.80290755, 4.4759332 , 5.84739223, 5.84236351],
       [1.21475352, 3.29399768, 1.69136631, 0.29040966, 1.71086031],
       [0.72581411, 3.21806371, 1.54808703, 0.36159148, 1.21567622]])

X_new의 첫번째 샘플이 첫번째 센트로이드에서 1.494, 두번째 센트로이드에서 2.886... 이렇게 떨어져 있다. 매우 효율적인 비선형 차원 축소 기법이 될 수 있다.

**K-평균 알고리즘**

어떻게 작동? 센트로이드가 주어지면 데이터셋에 있는 모든 샘플에 가장 가까운 센트로이드의 클러스터를 할당할 수 있다. 모든 샘플의 레이블이 주어져도 각 클러스터에 속한 샘플의 평균을 계산하여 모든 센트로이드를 쉽게 구할 수 있다. 하지만 주어지지 않으면? 센트로이드를 랜덤 설정한 뒤 샘플에 레이블 할당하고 센트로이드 업데이트 하고, 샘플에 레이블 할당하고 센트로이드를 업데이터 하는 식으로 센트로이드에 변화 없을때까지 계속한다. 

알고리즘 수렴을 보장하지만 적절한 솔루션으로 수렴하지는 못할 수도 있다. 이 여부는 센트로이드 초기화(랜덤하게 잡은 첫 센트로이드)에 달려 있다. 운. 센트로이드 초기화를 개선하여 이런 위험을 줄일 수 있는 방법을 보자.

**센트로이드 초기화 방법**

센트로이드 위치를 근사하게 알 수 있다면 init 매개변수에 센트로이드 리스트를 담은 넘파이 배열 지정, n_init=1로 설정할 수 있다.

In [8]:
good_init=np.array([[-3,3], [-3,2], [-3,1], [-1,2], [0,2]])
kmeans=KMeans(n_clusters=5, init=good_init, n_init=1)

In [9]:
kmeans.fit(X)

KMeans(init=array([[-3,  3],
       [-3,  2],
       [-3,  1],
       [-1,  2],
       [ 0,  2]]),
       n_clusters=5, n_init=1)

다른 방법은 랜덤 초기화를 다르게 하여 여러번 알고리즘을 실행하고 가장 좋은 솔루션을 선택하는 것. 랜덤 초기화 횟수는 n_init로 조절. 최선의 솔루션은 어떻게 알 수 있을까? by 성능 지표. 각 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리이며 모델의 **이너셔** 라고 한다. KMeans 클래스는 알고리즘을 n_init번 실행해 이너셔가 가장 낮은 모델을 반환. 이너셔 확인 가능하다.

In [10]:
kmeans.inertia_
#fit()을 수행한 뒤에 사용할 수 있다.

211.5985372581684

socre() 메서드는 이너셔의 음수값 반환. 큰 값이 좋은 것이라는 규칙 위해.

In [11]:
kmeans.score(X)

-211.59853725816836

k-평균++ 알고리즘.

다른 센트로이드와 거리가 먼 센트로이드를 선택하는 초기화 단계 소개. 최적이 아닌 솔루션으로 수렴할 가능성을 매우 낮춘다.
1. 무작위로 균등하게 하나의 센트로이드 선택
2. 확률에 따라 샘플을 새로운 센트로이드로 선택
3. k개의 센트로이드가 선택될 때까지 반복

KMeans는 기본적으로 이 초기화 방법을 사용.

k-평균 속도 개선과 미니배치 k-평균

불필요한 거리 계산을 피해서 알고리즘의 속도를 높이자. -> 삼각 부등식 이용. 샘플과 센트로이드 사이의 거리를 위한 하한선과 상한선을 유지.

전체 데이터셋을 사용해 반복하지 않고 각 반복마다 미니배치를 사용해 센트로이드를 조금씩 이동하자. -> 알고리즘 속도 개선, 메모리에 들어가지 않는 대량의 데이터셋에 군집 알고리즘 적용 가능. MiniBatchKMeans.

In [12]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans=MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(n_clusters=5)

데이터셋이 메모리에 들어가지 않으면 가장 간단한 방법은 점진적 PCA처럼 memmap 클래스 사용하는 것, 또는 MiniBatchKMeans 클래스의 partial_fit() 메서드에 한번에 하나의 미니배치를 전달하는 것. 하지만 비추.

미니배치 k-평균 알고리즘이 일반 k-평균 알고리즘보다 훨씬 빠르나 이너셔는 일반적으로 조금 더 나쁘다. 특히 클러스터 개수 증가할 때.

최적의 클러스터 개수 찾기

클러스터 개수 어떻게 설정할까? 올바르지 않으면 나빠짐. 가장 작은 이너셔를 가진 모델? 간단하지 않음. k가 증가하면 줄어드니까. 이너셔 그래프를 k의 함수로 그렸을 때 생기는 엘보(급격히 줄어드는 지점)는 좋은 선택이 될 수 있음

하지만 이건 너무 엉성. 더 정확한 방법은 실루엣 점수=모든 샘플에 대한 실루엣 계수 평균. (b-a)/max(a,b). a는 클러스터 내부의 평균 거리, b는 가장 가까운 클러스터의 샘플까지 평균 거리. 실루엣 계수가 +!에 가까우면 자신의 클러스터 안에 잘 속해 있고 다른 클러스터와는 멀리 있다는 것. 0이면 경계. -1이면 잘못됨.

In [13]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

0.655517642572828

이너셔만 봤을 때는 드러나지 않은 정보를 줌.

모든 샘플의 실루엣 계수를 할당된 클러스터와 계숫값으로 정렬하여 실루엣 다이어그램을 그리면 더 많은 정보가 보인다.

###9.1.2 k-평균의 한계
k-평균은 속도 빠르고 확장 용이하나 최적이 아닌 솔루션을 피하려면 알고리즘을 여러 번 실행해야 하고 클러스터 개수도 지정해야한다. 또한 클러스터의 크기나 밀집도가 서로 다르거나 원형 아니면 잘 작동 안함. ex) 타원형은 가우시안 혼합 모델이 더 낫다.

군집에서 도움을 받을 수 있는 몇 가지 방법을 보자.

###9.1.3 군집을 사용한 이미지 분할
이미지 분할=이미지를 세그먼트 여러 개로 분할하는 작업.
- 시맨틱 분할 : 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당.

시맨틱 또는 인스턴스 분할에서 최고 수준의 성능을 내기 위해선 합성곱 신경망을 사용한 복잡한 모델을 써야 하지만, 색상 분할을 우선 수행해보자. 동일한 색상을 가진 픽셀을 같은 세그먼트에 할당할 것이다.

In [14]:
import os
import urllib.request

In [15]:
# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

In [16]:
# 무당벌레 이미지를 다운로드합니다
images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
filename = "ladybug.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

('./images/unsupervised_learning/ladybug.png',
 <http.client.HTTPMessage at 0x7f1c582e0040>)

In [17]:
#먼저 맷플롯립의 imread() 함수로 이미지로 읽어보자
from matplotlib.image import imread #or 'from imageio import imread'
image=imread(os.path.join('images', 'unsupervised_learning', 'ladybug.png'))
image.shape

(533, 800, 3)

3D 배열. 높이x너비x컬러 채널 개수. 각 픽셀에 대해 빨초파 강도를 담은 3D 벡터가 있다. 0과 1 사이. 더 적은 채널을 갖는 흑백 이미지나 더 많은 채널을 갖는 위성 이미지 또는 알파 채널을 가진 이미지.

이 배열을 RGB 색상의 긴 리스트로 변환한 다음 k-평균 사용해 이 색상을 클러스터로 모으자.

In [18]:
X = image.reshape(-1, 3)
kmeans = KMeans(n_clusters=8, random_state=42).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


예를 들어 초록색을 하나의 컬러 클러스터로 모음. 그 담에 각 색상에 대해 그 픽셀의 컬러 클러스터의 평균 컬러를 찾는다. 마지막으로 이 긴 색상의 리스트를 원본 이미지와 동일한 크기로 바꾸면 완성.

클러스터 개수를 작게 하면 무당벌레의 빨간색이 독자적 클러스터 만들지 못하고 주위 색에 합쳐진다.

###9.1.4 군집을 사용한 전처리
군집은 차원 축소에 효과적. 지도 학습 알고리즘을 적용하기 전에 전처리 단계로 사용 가능. 데이터 불러오자.

In [19]:
from sklearn.datasets import load_digits
X_digits, y_digits=load_digits(return_X_y=True)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_digits, y_digits)

In [21]:
#로지스틱 훈련
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
# 정확도 평가
log_reg.score(X_test, y_test)

0.9688888888888889

k-평균을 전처리 단계로 사용하면 더 좋아지는지 보자.

파이프라인을 만들어 먼저 훈련 세트를 50개 클러스터로 모은다. 그 다음 이미지를 50개 클러스터까지 거리로 바꾸고 로지스틱 회귀 모델을 적용하자.

In [23]:
from sklearn.pipeline import Pipeline
pipeline=Pipeline([
    ('kmeans', KMeans(n_clusters=50)),
    ('log_reg', LogisticRegression())
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [24]:
#이 분류 파이프라인 평가
pipeline.score(X_test, y_test)

0.96

책에 따르면 오차율이 줄었다.

하지만 클러스터 개수를 임의로 정했다. 좋은 k값 찾으면 더 좋아질 것. 실루엣 분석을 하거나 이너셔가 감소되는지 확인 안해도 되고 교차 검증에서 가장 좋은 분류 선응을 내는 값을 찾으면 됨.

In [25]:
from sklearn.model_selection import GridSearchCV
param_grid=dict(kmeans__n_clusters=range(2,100))
grid_clf=GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.8s


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

[CV] END ..............................kmeans__n_clusters=99; total time=   2.4s


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ..............................kmeans__n_clusters=99; total time=   1.9s


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [26]:
#최선의 k값과 이때 파이프라인의 성능을 보자
grid_clf.best_params_

{'kmeans__n_clusters': 92}

In [27]:
grid_clf.score(X_test, y_test)

0.9577777777777777

정확도 향상.

###9.1.5 군집을 사용한 준지도 학습
군집을 사용하는 다른 예시는 준지도 학습. 레이블 없는 데이터 많고 레이블 있는 데이터는 적을 때 사용.

숫자 데이터셋에서 레이블된 50개 샘플에 로지스틱 회귀 훈련해보자.

In [28]:
n_labeled=50
log_reg=LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [29]:
#이 모델의 성능은?
log_reg.score(X_test, y_test)

0.8577777777777778

정확도 꽤 낮다. 어떻게 개선?

훈련 세트를 50개 클러스터로 모으고 각 클러스터에서 센트로이드에 가장 가까운 이미지를 찾는다. 대표 이미지.

In [30]:
k=50
kmeans=KMeans(n_clusters=k)
X_digits_dist=kmeans.fit_transform(X_train)
representative_digit_idx=np.argmin(X_digits_dist, axis=0)
X_representative_digits=X_train[representative_digit_idx]

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [31]:
y_train[representative_digit_idx]

array([5, 8, 3, 0, 6, 7, 4, 1, 5, 5, 2, 7, 4, 2, 9, 8, 2, 0, 5, 1, 3, 1,
       8, 2, 9, 7, 1, 7, 6, 1, 8, 0, 2, 6, 4, 4, 3, 0, 7, 9, 4, 1, 7, 9,
       5, 8, 1, 8, 2, 6])

In [71]:
#대표 이미지 50개를 보고 수동으로 레이블을 할당해보자
y_representative_digits = np.array([5, 8, 3, 0, 6, 7, 4, 1, 5, 5, 2, 7, 4, 2, 9, 8, 2, 0, 5, 1, 3, 1,
       8, 2, 9, 7, 1, 7, 6, 1, 8, 0, 2, 6, 4, 4, 3, 0, 7, 9, 4, 1, 7, 9,
       5, 8, 1, 8, 2, 6])

이제 레이블 된 50개 샘플로 이루어진 데이터셋 준비됨. 성능 좋은지 확인부터 하자.

In [72]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9

꽤 올랐다. 무작위 샘플 대신 대표 샘플에 레이블을 할당하는 것은 좋은 방법.

이 레이블을 동일한 클러스터에 있는 모든 샘플로 전파하면? = 레이블 전파

In [73]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [74]:
#다시 훈련하고 성능 확인
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9288888888888889

어느 정도 올렸으나 놀라울 정도는 아님. 각 대표 샘플의 레이블을 동일한 클러스터의 모든 샘플에 전파해서 문제. 센트로이드와 가까운 샘플의 20%만 전파해보고 다시 확인해보자.

In [75]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [76]:
#다시 훈련해보자
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9333333333333333

굿. 레이블된 샘플 50개만으로 이 정확도를 얻음. 전파된 레이블이 실제로 좋았기 때문.

In [77]:
np.mean(y_train_partially_propagated == y_train[partially_propagated])

0.9930555555555556

* 능동 학습

모델과 훈련 세트를 지속적으로 향상하기 위해 능동 학습을 몇 번 반복할 수 있다. 불확실성 샘플링 방식.
1. 수집한 레이블된 샘플에서 모델 훈련. 이 모델 사용해 레이블되지 않은 모든 샘플에 대한 예측 생성.
2. 모델이 가장 불확실하게 예측한 샘플을 전문가에게 보내 레이블을 붙임
3. 레이블을 부여하는 노력만큼 성능이 향상되지 않을 때까지 반복

###9.1.6 DBSCAN
밀집된 연속적 지역을 클러스터로 정의.
1. 알고리즘이 각 샘플에서 작은 거리 내에 샘플이 몇개 있는지 센다. 엡실론-이웃.
2. 엡실론-이웃 내 적어도 min_samples개 샘플이 있다면 이를 핵심 샘플로 간주.
3. 핵심 샘플의 이웃에 있는 모든 샘플은 동일한 클러스터에 속한다. 이웃에는 다른 핵심 샘플이 포함될 수 있어서 핵심 샘플의 이웃의 이웃은 계속해서 하나의 클러스터를 형성한다.
4. 핵심 샘플이 아니고 이웃도 아닌 샘플은 이상치로 판단.

클러스터가 충분히 밀집되어 있고 밀집되지 않은 지역과 잘 구분될 때 좋은 성능을 낸다. 반달 모양 데이터셋 테스트 해보자.

In [78]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y=make_moons(n_samples=1000, noise=0.05)
dbscan=DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

모든 샘플의 레이블은 인스턴스 변수 labels_에 저장되어 있다.

In [79]:
dbscan.labels_

array([ 0,  1, -1, -1,  0,  8,  0,  2,  4,  3,  1,  4, -1,  0,  5,  1, -1,
        1,  6,  1,  1,  7,  8, -1,  2,  8, -1,  7,  0,  9, -1,  1,  6,  6,
        3,  6,  3,  8,  1,  1,  1,  2, 10,  8, 11,  7,  1,  8,  3,  0,  1,
        0,  4,  0,  3,  1,  8,  8,  7,  2,  0,  3,  1,  1,  3,  2, -1,  2,
        1, -1, -1,  1,  8, 10,  6, -1, 10,  7,  7,  6,  1,  3,  1,  3,  1,
        1,  3,  1,  1,  0,  8,  0,  0,  0,  3,  1,  0,  5, 10,  1, -1,  1,
        1, 10,  3,  1,  3,  0,  8,  0,  0, 11,  3,  1,  3, -1,  0,  3,  0,
       -1,  0,  2,  9,  6,  0,  1,  9,  6,  4,  6,  3,  3,  1,  3,  1,  1,
        1,  9,  4,  6, -1,  1,  6, -1,  6,  6,  1,  8,  6,  6,  4,  6, 10,
        1,  0, -1,  3,  2,  1,  7,  9,  8,  3,  2,  6,  1,  6,  6,  1,  1,
        6,  0,  1,  1,  2,  6,  8,  2,  1,  0,  1,  6,  1,  2,  0,  6,  1,
        9,  4,  0,  2,  3,  4,  0, -1,  6,  1,  5,  3,  6, 10,  1, -1,  1,
        5,  1,  0,  6, -1,  1,  0,  8,  3,  6,  4,  8,  0,  1, -1,  6, 11,
        6,  1,  0,  7,  3

일부 샘플의 클러스터 인덱스는 -1 = 알고리즘이 이 샘플을 이상치로 판단했다. 

In [80]:
#핵심 샘플의 인덱스는 인스턴스 변수 core_samples_indices에서 확인 가능
len(dbscan.core_sample_indices_)

782

In [81]:
dbscan.core_sample_indices_

array([  0,   1,   4,   6,   7,   9,  10,  11,  13,  14,  15,  17,  18,
        20,  21,  22,  24,  25,  27,  28,  29,  32,  33,  34,  35,  36,
        37,  38,  39,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  58,  59,  61,  62,  63,  64,  65,
        67,  68,  71,  72,  73,  74,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  94,  95,  96,  97,
        98,  99, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112,
       113, 114, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127, 128,
       129, 130, 132, 133, 134, 135, 136, 137, 138, 139, 142, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159,
       160, 161, 164, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176,
       177, 178, 179, 180, 181, 182, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 195, 196, 197, 198, 199, 200, 201, 203, 204, 206,
       210, 211, 212, 213, 214, 215, 216, 217, 221, 222, 223, 22

In [82]:
#핵심 샘플 자체는 인스턴스 변수 components_에 저장
dbscan.components_

array([[-0.03116103,  0.43498187],
       [ 1.13716023, -0.43956071],
       [ 0.27297132, -0.17047278],
       ...,
       [ 1.69130787, -0.18948796],
       [ 1.55919695, -0.31196562],
       [ 0.95021523,  0.34219872]])

군집 결과 그래프를 보니 클러스터를 7개 만들었고 많은 샘플이 이상치가 됨. eps를 증가시키고 샘플의 이웃 범위 넓히면 완벽한 군집을 얻을 수 있다.

DBSCAN은 predict 제공 안함. 대신 fit_predict() 제공. -> 새로운 샘플에 대해 클러스터를 예측할 수 없다. 따라서 사용자가 필요한 예측기를 선택해야 한다.

In [83]:
#예를 들어 KNeighborsClassifier를 훈련
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

샘플 몇개를 전달해 어떤 클러스터에 속할 가능성이 높은지 예측하고 각 클러스터에 대한 확률 추정해보자.

In [84]:
X_new=np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2,1]])
knn.predict(X_new)

array([ 9,  0,  4, 10])

In [85]:
knn.predict_proba(X_new)

array([[0.14, 0.  , 0.  , 0.14, 0.  , 0.3 , 0.  , 0.  , 0.  , 0.42, 0.  ,
        0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.4 , 0.  , 0.  , 0.6 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.1 , 0.  , 0.  , 0.9 ,
        0.  ]])

이 분류기를 핵심 샘플에서만 훈련했지만 모든 샘플에서 훈련할 수도 있다. 이상치를 제외할 수도 있다.

이 결정 경계의 그림을 보자. 훈련 세트에 이상치가 없기 때문에 클러스터가 멀리 떨어져 있더라도 분류기는 항상 클러스터 한 개를 선택한다. 최대 거리를 사용하면 두 클러스터에서 멀리 떨어진 샘플을 이상치로 간단히 분류할 수 있다. KNeighborsClassifier의 kneighbors() 메서드를 이용한다. 여기에 샘플을 전달하면 훈련 세트에서 가장 가까운 k개 이웃의 거리와 인덱스를 반환한다.

In [86]:
y_dist, y_pred_idx=knn.kneighbors(X_new, n_neighbors=1)
y_pred=dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist>0.2]=-1
y_pred.ravel()

array([-1,  0,  4, -1])

dbscan은 간단하지만 강력. 클러스터의 모양과 개수에 상관 없이 감지할 수 있다. 이상치에 안정적, 하이퍼 파라미터도 두개뿐. 하지만 클러스터 간의 밀집도가 크게 다르면 모든 클러스터를 올바르게 잡아내는 것이 불가능. 계산 복잡도는 O(m log m)이다. eps가 커지면 O(m^2)만큼 메모리가 필요. 

###9.1.7 다른 군집 알고리즘

**병합 군집**

클러스터 계층을 밑바닥부터 위로 쌓아 구성. 인접한 클러스터 쌍을 연결해서 병합된 클러스터 상을 트리로 모두 그리면 클러스터의 이진 트리를 얻을 수 있다. 이 트리의 리프는 개별 샘플. 병합 군집은 대규모 샘플과 클러스터에 잘 확장되며 다양한 형태의 클러스터를 감지할 수 있다. 특정 클러스터 개수를 선택하는 데 도움이 되는 유용한 클러스터 트리를 만들 수도 있다. 어떤 짝 거리와도 사용 가능. 연결 행렬이 없으면 대규모 데이터셋으로 확장이 어렵다.

**BIRCH**

특별히 대규모 데이터셋을 위해 고안됨. 특성 개수가 너무 많지 않다면 좋다. 새로운 샘플을 클러스터에 빠르게 할당할 수 있는 정보를 담은 트리 구조를 만든다. 이 트리에 모든 샘플을 저장하지 않는다.

**평균-이동**

각 샘플을 중심으로 하는 원을 그려 원마다 안에 포함된 모든 샘플의 평균을 구한다. 그리고 원의 중심을 평균점으로 이동시키고 반복한다. 지역의 최대 밀도를 찾을 때까지 높은 쪽으로 원을 이동. 동일 지역에 안착한 원에 있는 모든 샘플은 동일한 클러스터가 된다. 모양이나 개수에 상관 없이 클러스터를 찾을 수 있다. 하이퍼 파라미터도 매우 적다. 평균-이동은 클러스터 내부 밀집도가 불균형할 때 여러 개로 나누는 경향이 있다. 대규모 데이터셋에는 적합하지 않다.

**유사도 전파**

투표 방식 사용. 샘플은 자신과 비슷한 샘플에 투표. 각 대표와 투표한 샘플이 클러스터 형성. 크기가 다른 여러 개의 클러스터 감지 가능. 대규모 데이터셋에는 적합하지 않다.

**스펙트럼 군집**

샘플 사이의 유사도 행렬을 받아 저차원 임베딩을 만든다(차원 축소). 이 저차원 공간에서 또 다른 군집 알고리즘 사용. 복잡한 클러스터 구조를 감지하고 그래프 컷을 찾는데 사용할 수 있다. 샘플 개수가 많거나 클러스터의 크기가 매우 다르면 잘 작동하지 않는다.

##9.2 가우시안 혼합
샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델 = 가우시안 혼합 모델. 하나의 가우시안 분포에서 생성된 모든 샘플은 하나의 타원형 클러스터 형성. 각 클러스터의 모양, 크기, 밀집도 등등은 다름. 샘플이 주어지면 가우시안 분포 중 하나에서 생성되었다는 것을 알지만 파라미터는 모름.

여러 GMM 변종이 있음. 가장 간단한 버전이 GaussianMixture 클래스로 구현 가능. 사전에 가우시안 분포의 개수 k를 알아야한다.

데이터셋 X 생성 과정은 그래프 모형으로 나타낼 수 있다. 이 모델로 데이터셋 X가 주어지면 가중치와 전체 분포의 파라미터를 추정.

In [87]:
from sklearn.mixture import GaussianMixture

gm=GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [102]:
#이 알고리즘이 추정한 파라미터
gm.weights_

array([0.21650302, 0.20566744, 0.57782954])

In [89]:
gm.means_

array([[-0.70640157,  0.57975835],
       [ 1.74773153, -0.06380322],
       [ 0.51105218,  0.23613702]])

In [90]:
gm.covariances_

array([[[ 0.06937074,  0.07304433],
        [ 0.07304433,  0.09368442]],

       [[ 0.05622215,  0.06439513],
        [ 0.06439513,  0.08745777]],

       [[ 0.15939369, -0.09124289],
        [-0.09124289,  0.2834946 ]]])

잘 작동했다. 실제 데이터에서 사용한 가중치는 0.2, 0.4, 0.4임. 평균, 분산행렬도 비슷. 

어떻게 한 것일까? 기댓값-최대화 알고리즘 사용. 샘플을 클러스터에 할당하고(기댓값 단계) 클러스터를 업데이터한다(최대화 단계). k-평균과 비슷. 하지만 EM은 하드 클러스터 할당이 아니라 소프트 클러스터 할당을 사용한다. 최대화 단계에서 클러스터 업데이트는 책임(클러스터에 속할 추정 확률)이 가장 많은 샘플에 크게 영향을 받는다.

알고리즘이 수렴했는지 여부와 반복 횟수를 확인할 수 있다.

In [91]:
gm.converged_

True

In [92]:
gm.n_iter_

13

여기까지 각 클러스터의 위치, 크기, 모양, 방향, 상대적 가중치 예측함. 이 모델은 새로운 샘플을 가장 비슷한 클러스터에 쉽게 할당 가능하거나 특정 클러스터에 속할 확률을 예측할 수 있다.

In [93]:
#하드군집을 위해서는
gm.predict(X)

array([2, 2, 1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 1, 0, 2, 2, 1, 2, 1,
       2, 2, 2, 0, 2, 1, 2, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 1, 2, 2, 1, 0,
       2, 1, 1, 0, 0, 2, 2, 2, 2, 2, 0, 1, 0, 0, 1, 2, 2, 0, 1, 2, 0, 2,
       2, 2, 1, 2, 1, 2, 0, 1, 2, 0, 1, 1, 1, 2, 2, 0, 2, 0, 2, 1, 0, 1,
       2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2,
       2, 2, 0, 1, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 0, 2, 2, 2, 0, 2, 2, 1, 2, 2, 1, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2,
       2, 0, 0, 2, 1, 1, 0, 2, 0, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 2, 1, 2, 2, 0,
       0, 2, 1, 1, 2, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2,
       2, 2, 2, 2, 1, 0, 1, 2, 2, 2, 1, 2, 2, 0, 2, 1, 0, 2, 0, 1, 1, 1,
       2, 2, 2, 2, 0, 0, 1, 0, 1, 1, 2, 2, 2, 2, 0, 1, 2, 1, 1, 2, 2, 1,
       2, 2, 1, 2, 2, 1, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0,
       1, 1, 0, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2,

In [94]:
#소프트군집을 위해서는
gm.predict_proba(X)

array([[6.71843272e-011, 4.30351612e-114, 1.00000000e+000],
       [6.83145311e-124, 8.26275923e-003, 9.91737241e-001],
       [5.02358938e-161, 9.99877836e-001, 1.22163666e-004],
       ...,
       [3.39742354e-159, 9.94735528e-001, 5.26447151e-003],
       [8.93634110e-156, 9.81997353e-001, 1.80026472e-002],
       [1.82385758e-059, 5.32333558e-030, 1.00000000e+000]])

가우시안 혼합 모델은 생성 모델. 즉, 이 모델에서 새로운 샘플을 만들 수 있다.

In [95]:
X_new, y_new=gm.sample(6)
X_new

array([[-0.21721719,  1.2617064 ],
       [ 1.94151812,  0.10724854],
       [ 0.80033442, -0.46093437],
       [ 1.15593576, -1.09833998],
       [ 0.74192879,  0.14949745],
       [-0.18782074,  0.0265524 ]])

In [96]:
y_new

array([0, 1, 2, 2, 2, 2])

또한 주어진 위치에서 모델의 밀도를 추정할 수 있다 (score_samples()). 샘플 주어지면 이 메서드는 그 위치의 확률 밀도 함수의 로그를 예측한다. 점수 높을수록 밀도 높다.

In [97]:
gm.score_samples(X)

array([-1.68519318e+00, -2.17512268e+00, -7.13051151e-01, -8.84734786e-01,
       -1.55110241e+00, -1.66115112e+00, -1.49377824e+00, -1.72391427e+00,
       -1.44851963e+00, -3.75507996e-01, -2.20889952e+00, -1.49052430e+00,
       -1.90442360e+00, -1.36920457e+00, -3.06791426e-01, -4.86088814e-01,
       -6.44385492e-01, -1.51405076e+00, -1.33853276e+00, -1.02341826e+00,
       -1.84232944e+00, -2.01976352e-01, -2.01103290e+00, -2.29611548e+00,
       -1.57017237e+00, -6.64147449e-01, -2.37836042e+00, -2.13952060e-02,
       -1.56403128e+00, -2.14469907e+00, -1.33681079e+00, -7.22040285e-01,
       -1.43577284e+00, -1.73477556e+00, -2.27558733e-01, -1.52459451e+00,
       -2.15395116e-01, -2.06180799e+00, -1.62534854e+00, -5.19559047e-01,
       -1.55327911e+00, -1.84451000e+00, -1.27748749e+00, -8.97945044e-01,
       -1.94903125e+00, -2.80141802e-01,  9.63531157e-02, -6.05719837e-01,
       -7.33967760e-01, -1.50484538e+00, -1.66048493e+00, -1.34513473e+00,
       -1.51179476e+00, -

점수의 지수값 계산하면 샘플의 위치에서 pdf를 알 수 있겠다. 하지만 확률이 아니라 확률 밀도이므로 샘플이 특정 지역 안에 속할 확률을 예측하려면 적분해야한다.

하지만 실제 데이터 사용시 가우시안 분포나 저차원이 아닌 경우가 많고, 이 알고리즘에는 정확한 클러스터 개수도 입력했다. 특성, 클러스터 많거나 샘플 적으면 EM이 최적의 솔루션으로 수렴하기 어렵다. 이런 경우 알고리즘이 학습할 파라미터 개수를 제한해야함. 클러스터의 모양과 방향의 범위 제한하기. covariance_type 매개변수에 'spherical', 'diag', 'tied' 중 하나를 설정하기.

가우시안 혼합 모델은 이상치 탐지에도 사용할 수 있다.

###9.2.1 가우시안 혼합을 사용한 이상치 탐지
이상치는 부정 거래 감지, 제조 결함이 있는 제품 감지에 사용. 또는 이상치 제거에 사용. 가우시안 혼합 모델을 이상치 탐지에 사용하는 방법은 매우 간단. 밀도가 낮은 지역에 있는 모든 샘플을 이상치로 볼 수 있음. 이렇게 하려면 사용할 밀도 임계값을 정해야한다. ex) 제조 회사에서 결함 제품의 비율=4%로 알고 있다면 밀도 임계값을 이 값으로 설정하면 밀도가 낮은 지역에 있는 샘플의 4%를 얻을 수 있다.

4%를 밀도 임계값으로 정해 이상치를 구분하는 방법을 보자.

In [98]:
densities=gm.score_samples(X)
density_threshold=np.percentile(densities, 4)
anomalies=X[densities<density_threshold]

이와 비슷한 작업은 특이치 탐지. 이상치로 오염되지 않은 깨끗한 데이터셋에서 훈련한다는 것이 이상치 탐지와 다르다.

k-평균처럼 GaussianMixture 알고리즘은 클러스터의 개수를 지정해야한다. 어떻게 찾을까?

###9.2.2 클러스터 개수 선택하기
k-평균에서는 이너셔나 실루엣 점수를 사용해 클러스터 개수 선택. 이런 지표는 클러스터가 타원형이거나 크기가 다를 때 안정적이지 않아서 가우시안에서 사용 불가. 대신 BIC, AIC, 이론적 정보 기준을 최소화하는 모델을 찾는다. BIC, AIC는 파라미터 많은=클러스터 많은 모델에게 벌칙 부과. 하지만 데이터에 아주 잘 맞지는 않을 수 있다. 특히 대규모 데이터셋에서. 

- 가능도 함수

확률=/=가능도. 파라미터=세타인 확률 모델이 주어지면 확률은 미래 출력 x가 얼마나 그럴듯한지 설명, 가능도는 출력 x를 알고 있을 때 특정 파라미터 값=세타가 얼마나 그럴듯한지 설명한다. pdf는 x의 함수, 가능도는 세타의 함수 (x는 고정).

데이터셋 X가 주어졌을 때 일반적으로 모델 파라미터에 대해 가장 그럴듯한 값이 예측. 이를 위해 X에 대한 가능도 함수를 최대화하는 값을 찾아야한다. 최대 가능도 추정(MLE=theta_hat), 최대 사후 확률. 가능도 함수를 최대화 하는 것은 최대 사후 확률의 로그를 최대화 하는 것과 같다. MLE를 추정하고 나면 AIC, BIC를 계산하기 쉬워진다.

In [99]:
#AIC, BIC 계산
gm.bic(X)

2754.718884004445

In [100]:
gm.aic(X)

2671.2870442617486

그림을 보면 k=3에서 AIC, BIC가 모두 가장 작으므로 k=3 선택. 

###9.2.3 베이즈 가우시안 혼합 모델
최적의 클러스터 개수를 수동으로 찾지 않고 불필요한 클러스터의 가중치를 0으로 만드는 BayesianGaussianMixture 클래스를 사용할 수 있다. 클러스터 개수 n_components를 최적의 클러스터 개수보다 크다고 믿을 만한 값으로 지정한다. 자동으로 불필요한 클러스터를 제거해준다.

In [101]:
#클러스터 개수 10으로 설정하고 결과를 보자
from sklearn.mixture import BayesianGaussianMixture
bgm=BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_, 2)

array([0.2 , 0.12, 0.14, 0.16, 0.12, 0.12, 0.  , 0.  , 0.  , 0.14])

알고리즘이 자동으로 7개의 클러스터가 필요하다고 감지함.

이 모델에서 클러스터 파라미터는 더는 고정된 모델 파라미터가 아니라 클러스터 할당처럼 잠재 확률 변수로 취급된다.

베타분포는 고정 범위 안에 놓인 값을 가진 확률 변수를 모델링할 때 자주 사용된다. ex) SBP. 이 프로세스는 새로운 샘플이 작은 클러스터보다 큰 클러스터에 합류할 가능성이 높은 데이터셋에 잘 맞는 모델이다.

잠재 변수 z에 대한 사전 지식이 사전 확률이라는 확률 분포에 인코딩 될 수 있다. 사전 믿음. 클러스터 개수에 대한 사전 믿음은 weight_concentration_prior 매개변수로 조절 가능. 하지만 데이터 많을수록 사전 믿음은 중요하지 않다.

베이즈 정리는 데이터 X를 관측하고 난 후 잠재 변수에 대한 확률 분포를 업데이트 하는 방법을 설명. 사후 확률분포를 계산. 하지만 가우시안 혼합 모델에서 증거 확률을 계산하기가 힘들다. 이 문제를 해결하기 위한 방법 하나가 변분 추론. 자체적인 변분 파라미터 람다를 가진 분포 패밀리를 선택한다. 그 다음 분포 패밀리가 사후확률의 좋은 근삿값이 되도록 이 파라미터 최적화. KL 발산. 실전에서는 다른 기법으로 ELBO를 최대화. 평균장 변분 추론에서 ELBO 식을 계산할 수 있는 형태로 단순화하기 위해 분포 패밀리의 사전 확률을 주의 깊게 선택할 것. 수학적 기술 필요. ELBO를 최대화하는 간단한 방법은 블랙 박스 확률적 변분 추론. 이 방법은 어떤 종류의 모델과도 베이즈 추론을 사용할 수 있게 한다. 심층 신경망도 가능. -> 딥러닝.

가우시안 혼합 모델은 타원형에는 잘 작동하지만 다른 모양을 가진 데이터셋에 훈련하면 나쁜 결과를 얻을 것. 임의의 모양을 가진 클러스터는 어떤 군집 알고리즘 사용?

###9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘
- PCA (그리고 inverse_transform() 메서드를 가진 다른 차원 축소 기법)
- Fast-MCD : 데이터셋 정제 시 사용. 보통 샘플이 혼합된 것이 아니라 하나의 가우시안 분포에서 생성되었다고 가정하며, 가우시안 분포에서 생성되지 않은 이상치로 이 데이터셋이 오염되었다고 가정. 알고리즘이 가우시안 분포의 파라미터를 추정할 때 이상치로 의심되는 샘플을 무시. 이 기법은 알고리즘이 타원형을 잘 추정하고 이상치를 잘 구분하도록 돕는다.
- 아이솔레이션 포레스트 : 고차원 데이터셋에서 유용. 무작위로 성장한 결정 트리로 구성된 랜덤 포레스틀 만든다. 각 노드에서 특성을 랜덤하게 선택한 다음 랜덤 임계값 골라 데이터를 두개로 나눈다. 모든 샘플이 다른 샘플과 격리 될 때까지 반복. 이상치는 보통 다른 샘플과 멀리 떨어져 있어서 평균적으로 정상 샘플과 적은 단계에서 격리 가능.
- LOF : 샘플 주위의 밀도와 이웃 주위의 밀도를 비교. 이상치는 k개의 최근접 이웃보다 더 격리된다.
- one-class SVM : 특이치 탐지. 모든 샘플을 고차원 공간에 매핑한 다음 이 공간에서 선형 SVM 분류기로 분류. 샘플의 클래스가 하나이기 때문에 대신 one-class SVM 알고리즘이 원본 공간으로부터 고차원 공간에 있는 샘플 분리. 원본 공간에서는 모든 샘플을 둘러싼 작은 영역을 찾는 것에 해당. 새로운 샘플이 이 영역 안에 놓이지 않으면 이상치. 하이퍼 파라미터 적다. 하지만 대규모 데이터셋으로 확장은 어렵다.